In [ ]:
import cv2
import numpy as np
import math
from object_detection import ObjectDetection

In [ ]:
cap = cv2.VideoCapture('road_tracking.mp4')
object_detector = cv2.createBackgroundSubtractorMOG2(detectShadows=True) #history=100, varThreshold=100
threshold = 900

count = 0

tracking_obj = {}
tracking_id = 0
center_points_prev_frame = []
while True:
    succ, frame = cap.read()
    # print(frame.shape[0:2])
    count += 1

    if not succ:
        break
    roi = frame[300:, 100:1000] # height , width

    center_points_curr_frame = []

    # object detection part ------------------------------------------
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 250, 255, cv2.THRESH_BINARY)
    mask = cv2.erode(mask,None, iterations=1)
    mask = cv2.dilate(mask,None, iterations=2)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for ctn in contours:
        area = cv2.contourArea(ctn)

        if area > threshold:
            # cv2.drawContours(roi, [ctn], -1, (0,255,0), 2)
            x, y, w, h = cv2.boundingRect(ctn)

            cx = int((x + x + w) / 2)
            cy = int((y + y + h) / 2)
            center_points_curr_frame.append((cx, cy))
            
            cv2.rectangle(roi, (x,y), (x + w, y + h), (0,255,0), 2)

    mask = cv2.bitwise_and(roi, roi, mask=mask)
    #-------------------------------------------------------------------
    # object tracking part ---------------------------------------------
    if count <= 2:

        for pt1 in center_points_curr_frame:
            for pt2 in center_points_prev_frame:
                distance = math.hypot(pt2[0] - pt1[0], pt2[1] - pt1[1])

                if distance < 20:
                    tracking_obj[tracking_id] = pt1
                    tracking_id += 1

    else:
        object_tracking_copy = tracking_obj.copy()
        center_points_curr_frame_copy = center_points_curr_frame.copy()

        for object_id, pt2 in object_tracking_copy.items():
            object_exists = False
            for pt1 in center_points_curr_frame_copy:
                distance = math.hypot(pt2[0] - pt1[0], pt2[1] - pt1[1])
                if distance < 20:
                    tracking_obj[object_id] = pt1
                    object_exists = True

                    if pt1 in center_points_curr_frame:
                        center_points_curr_frame.remove(pt1)
                    continue

            if not object_exists:
                tracking_obj.pop(object_id)

        for pt in center_points_curr_frame:
            tracking_obj[tracking_id] = pt
            tracking_id += 1

    for object_id, pt in tracking_obj.items():
        cv2.circle(roi, pt, 5, (0, 0, 255), -1)
        cv2.putText(roi, '{}'.format(object_id), (pt[0] , pt[1] - 10), 0 , 1, (0, 0, 255), 2)
    #-------------------------------------------------------------------
    cv2.imshow('roi', roi)
    cv2.imshow('frame',frame)
    cv2.imshow('mask', mask)

    center_points_prev_frame = center_points_curr_frame.copy()


    key = cv2.waitKey(25)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture('road_tracking.mp4')
od = ObjectDetection()

count = 0

tracking_obj = {}
tracking_id = 0
center_points_prev_frame = []
while True:
    succ, frame = cap.read()
    # print(frame.shape[0:2])
    count += 1

    if not succ:
        break
    roi = frame[300:, 100:1000] # height , width

    center_points_curr_frame = []

    # object detection part ------------------------------------------
    _, _, boxes = od.detect(roi)
    for box in boxes:
        x, y, w, h = box
        cx = int((x + x + w) / 2)
        cy = int((y + y + h) / 2)
        center_points_curr_frame.append((cx, cy))
        cv2.rectangle(roi, (x,y), (x + w, y + h), (0,255,0), 2)
    #-------------------------------------------------------------------
    # object tracking part ---------------------------------------------
    if count <= 2:

        for pt1 in center_points_curr_frame:
            for pt2 in center_points_prev_frame:
                distance = math.hypot(pt2[0] - pt1[0], pt2[1] - pt1[1])

                if distance < 20:
                    tracking_obj[tracking_id] = pt1
                    tracking_id += 1

    else:
        object_tracking_copy = tracking_obj.copy()
        center_points_curr_frame_copy = center_points_curr_frame.copy()

        for object_id, pt2 in object_tracking_copy.items():
            object_exists = False
            for pt1 in center_points_curr_frame_copy:
                distance = math.hypot(pt2[0] - pt1[0], pt2[1] - pt1[1])
                if distance < 20:
                    tracking_obj[object_id] = pt1
                    object_exists = True

                    if pt1 in center_points_curr_frame:
                        center_points_curr_frame.remove(pt1)
                    continue

            if not object_exists:
                tracking_obj.pop(object_id)

        for pt in center_points_curr_frame:
            tracking_obj[tracking_id] = pt
            tracking_id += 1

    for object_id, pt in tracking_obj.items():
        cv2.circle(roi, pt, 5, (0, 0, 255), -1)
        cv2.putText(roi, '{}'.format(object_id), (pt[0] , pt[1] - 10), 0 , 1, (0, 0, 255), 2)
    #-------------------------------------------------------------------
    cv2.imshow('roi', roi)
    cv2.imshow('frame',frame)

    center_points_prev_frame = center_points_curr_frame.copy()


    key = cv2.waitKey(25)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import sys

(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
print(cv2.__version__)

if __name__ == '__main__':

    # Set up tracker.
    # Instead of MIL, you can also use

    tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'CSRT', 'MOSSE']
    tracker_type = tracker_types[6]

    if int(major_ver) < 4 and int(minor_ver) < 3:
        tracker = cv2.legacy.Tracker_create(tracker_type)
    else:
        if tracker_type == 'BOOSTING':
            tracker = cv2.legacy.TrackerBoosting_create()
        if tracker_type == 'MIL':
            tracker = cv2.legacy.TrackerMIL_create()
        if tracker_type == 'KCF':
            tracker = cv2.legacy.TrackerKCF_create()
        if tracker_type == 'TLD':
            tracker = cv2.legacy.TrackerTLD_create()
        if tracker_type == 'MEDIANFLOW':
            tracker = cv2.legacy.TrackerMedianFlow_create()
        if tracker_type == 'CSRT':
            tracker = cv2.legacy.TrackerCSRT_create()
        if tracker_type == 'MOSSE':
            tracker = cv2.legacy.TrackerMOSSE_create()

    # Read video
    video = cv2.VideoCapture('road_tracking.mp4')

    # Exit if video not opened.
    if not video.isOpened():
        print("Could not open video")
        sys.exit()

    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print('Cannot read video file')
        sys.exit()

    # Define an initial bounding box
    bbox = (287, 23, 86, 320)

    # Uncomment the line below to select a different bounding box
    bbox = cv2.selectROI(frame, False)

    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)

    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break

        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        ok, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);

        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);

        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

        # Display result
        cv2.imshow("Tracking", frame)

        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27 : break

4.4.0


AttributeError: module 'cv2.cv2' has no attribute 'legacy'